In [6]:
# Cell 1: We will need the RBCPath type from the rbclib package to load data from the RBC.
from rbclib import RBCPath

# We'll also want to load some data directly from the filesystem.
from pathlib import Path

# We'll want to load/process some of the data using pandas and numpy.
import pandas as pd
import numpy as np

In [7]:
# Cell 2 (updated)
# Participant meta-data is generally located in the BIDS repository for each study:
rbcdata_path = Path('/home/jovyan/shared/data/RBC')
train_filepath = rbcdata_path / 'train_participants.tsv'
test_filepath  = rbcdata_path / 'test_participants.tsv'

# Load the PNC participants TSV files...
with train_filepath.open('r') as f:
    train_data = pd.read_csv(f, sep='\t')
with test_filepath.open('r') as f:
    test_data = pd.read_csv(f, sep='\t')

# Combine into a single dataframe of all study participants:
all_data = pd.concat([train_data, test_data], ignore_index=True)

# 1) your known bad participants (missing multiple DKT ROIs)
participants_to_drop = {
    '1342487188', '1649551035', '2003542642', '219325366', '2249226316',
    '4184549693', '495793681', '4205323727', '533698126'
}

# 2) QC-determined bad participants (we will fill this later once we load the QC TSV)
qc_fail_participants = set()   # <-- placeholder, will update later

# Show the participant table (like the authors did)
all_data


,participant_id,study,study_site,session_id,wave,age,sex,race,ethnicity,bmi,handedness,participant_education,parent_1_education,parent_2_education,p_factor
0,1000393599,PNC,PNC1,PNC1,1,15.583333,Male,Black,not Hispanic or Latino,22.15,Right,9th Grade,Complete primary,Complete secondary,0.589907
1,1001970838,PNC,PNC1,PNC1,1,17.833333,Male,Other,Hispanic or Latino,23.98,Right,11th Grade,Complete tertiary,Complete tertiary,-0.659061
2,1007995238,PNC,PNC1,PNC1,1,13.750000,Female,Other,not Hispanic or Latino,23.77,Right,6th Grade,Complete tertiary,Complete primary,-1.608375
3,1011497669,PNC,PNC1,PNC1,1,16.666667,Male,White,not Hispanic or Latino,29.68,Right,9th Grade,Complete tertiary,Complete tertiary,-1.233807
4,1017092387,PNC,PNC1,PNC1,1,18.666667,Female,Black,not Hispanic or Latino,23.24,Right,11th Grade,Complete primary,Complete primary,-0.923100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,969649154,PNC,PNC1,PNC1,1,12.333333,Male,White,not Hispanic or Latino,17.38,Right,5th Grade,Complete tertiary,Complete secondary,NaN
1597,970890500,PNC,PNC1,PNC1,1,18.166667,Female,White,not Hispanic or Latino,30.89,Right,11th Grade,Complete secondary,Complete secondary,NaN
1598,975856179,PNC,PNC1,PNC1,1,11.000000,Male,White,not Hispanic or Latino,15.67,Right,4th Grade,Complete primary,Complete secondary,NaN
1599,984757368,PNC,PNC1,PNC1,1,13.416667,Male,Black,not Hispanic or Latino,16.66,Right,5th Grade,Complete primary,NaN,NaN


In [8]:
# Cell 3 (same logic, clearer comment)
def load_fsdata(participant_id, local_cache_dir=(Path.home() / 'cache')):
    """
    Loads and returns the FreeSurfer region-surface-stats TSV
    for a single PNC participant.
    We will call this for every participant later and then
    extract the aparc.DKTatlas rows we care about.
    """
    if local_cache_dir is not None:
        local_cache_dir = Path(local_cache_dir)
        local_cache_dir.mkdir(exist_ok=True)

    pnc_freesurfer_path = RBCPath(
        'rbc://PNC_FreeSurfer/freesurfer',
        local_cache_dir=local_cache_dir
    )
    participant_path = pnc_freesurfer_path / f'sub-{participant_id}'
    tsv_path = participant_path / f'sub-{participant_id}_regionsurfacestats.tsv'

    with tsv_path.open('r') as f:
        data = pd.read_csv(f, sep='\t')

    return data

In [9]:
# Cell 4: Summed (+ normalized) DKT GrayVol for a single participant

import pandas as pd
import numpy as np

def load_dkt_summed_and_normalized_grayvol(participant_id, local_cache_dir=(Path.home() / 'cache')):
    """
    Returns a pandas Series with:
      - total_dkt_grayvol
      - sum_grayvol_<StructName>  (lh + rh; if only one hemi present, uses that)
      - norm_grayvol_<StructName> (sum_grayvol_<StructName> / total_dkt_grayvol)
    Uses only atlas == 'aparc.DKTatlas'.
    """
    df = load_fsdata(participant_id, local_cache_dir=local_cache_dir)

    # Keep only DKT rows and the columns we need
    df = df[df['atlas'] == 'aparc.DKTatlas'][['StructName', 'hemisphere', 'GrayVol']].copy()

    if df.empty:
        # No DKT rows for this participant
        return pd.Series({'total_dkt_grayvol': np.nan}, dtype=float)

    # Sum across hemispheres per structure (lh + rh). If only one hemi exists, sum = that value.
    # min_count=1 ensures that if both hemispheres were missing, result is NaN (not 0).
    summed = df.groupby('StructName', as_index=True)['GrayVol'].sum(min_count=1)

    # Total DKT gray volume (sum over structures, ignoring NaNs)
    total = summed.sum(min_count=1)

    # Build Series with consistent names
    summed_named = summed.rename(lambda s: f"sum_grayvol_{s}")
    if pd.isna(total) or total <= 0:
        normalized_named = summed * np.nan
    else:
        normalized_named = (summed / total).rename(lambda s: f"norm_grayvol_{s}")

    out = pd.concat([pd.Series({'total_dkt_grayvol': float(total)}), summed_named, normalized_named])
    out.name = None
    return out

In [40]:
# Cell 4_AI: Compute AI for DKT GrayVol per participant
# AI = (rh - lh) / (rh + lh), NaN if a hemi is missing or denominator <= 0

import pandas as pd
import numpy as np
from pathlib import Path

def load_dkt_ai_grayvol(participant_id, local_cache_dir=(Path.home() / 'cache')):
    """
    Returns a pandas Series with:
      - ai_grayvol_<StructName>  ( (rh - lh) / (rh + lh) )
    Uses only atlas == 'aparc.DKTatlas'.
    """
    df = load_fsdata(participant_id, local_cache_dir=local_cache_dir)
    df = df[df['atlas'] == 'aparc.DKTatlas'][['StructName','hemisphere','GrayVol']].copy()
    if df.empty:
        return pd.Series(dtype=float)

    piv = df.pivot_table(index='StructName', columns='hemisphere', values='GrayVol', aggfunc='first')
    lh = piv.get('lh')
    rh = piv.get('rh')
    denom = lh + rh
    ai = (rh - lh) / denom
    ai.name = None
    ai = ai.rename(lambda s: f"ai_grayvol_{s}")
    return ai


In [41]:
# Cell 5: Preview for one example participant
example_participant_id = 1000393599  # same as authors' example
example_feats = load_dkt_summed_and_normalized_grayvol(example_participant_id)

# Show a few entries
example_feats.head(20)

total_dkt_grayvol                      541205.0
sum_grayvol_caudalanteriorcingulate      5585.0
sum_grayvol_caudalmiddlefrontal         13717.0
sum_grayvol_cuneus                      11162.0
sum_grayvol_entorhinal                   5090.0
sum_grayvol_fusiform                    17342.0
sum_grayvol_inferiorparietal            32167.0
sum_grayvol_inferiortemporal            25353.0
sum_grayvol_insula                      12772.0
sum_grayvol_isthmuscingulate             5131.0
sum_grayvol_lateraloccipital            27557.0
sum_grayvol_lateralorbitofrontal        19203.0
sum_grayvol_lingual                     14022.0
sum_grayvol_medialorbitofrontal         10319.0
sum_grayvol_middletemporal              30510.0
sum_grayvol_paracentral                  9907.0
sum_grayvol_parahippocampal              4701.0
sum_grayvol_parsopercularis              9138.0
sum_grayvol_parsorbitalis                5015.0
sum_grayvol_parstriangularis             8414.0
dtype: float64

In [11]:
# Cell 6: Build the full feature table (summed + normalized DKT GrayVol)
# - Keeps participants_to_drop defined in Cell 2 (we will filter later, after QC load)
# - Adds p_factor from all_data (NaN for test)
# - No QC filtering yet; we will merge/ filter after we load QC in a later cell.

print("Building DKT summed & normalized GrayVol features for all participants...")

from ipywidgets import IntProgress
from IPython.display import display

rows = []
prog = IntProgress(min=0, max=len(all_data))
display(prog)

for _, row in all_data.iterrows():
    pid = str(row['participant_id'])
    pval = row.get('p_factor', np.nan)

    try:
        s = load_dkt_summed_and_normalized_grayvol(pid)
    except FileNotFoundError:
        # Missing FS TSV for this participant
        s = pd.Series({'total_dkt_grayvol': np.nan}, dtype=float)

    rec = {'participant_id': pid, 'p_factor': pval}
    rec.update(s.to_dict())
    rows.append(rec)
    prog.value += 1

# Assemble the table
all_vars_vol = pd.DataFrame(rows)

# (Optional preview) How many feature columns we created?
sum_cols  = [c for c in all_vars_vol.columns if c.startswith('sum_grayvol_')]
norm_cols = [c for c in all_vars_vol.columns if c.startswith('norm_grayvol_')]
print(f"Feature columns: {len(sum_cols)} summed, {len(norm_cols)} normalized; + total_dkt_grayvol")

# Train/test split mirrors the authors' style
train_vars_vol = all_vars_vol[~np.isnan(all_vars_vol['p_factor'])].reset_index(drop=True)
test_vars_vol  = all_vars_vol[ np.isnan(all_vars_vol['p_factor'])].reset_index(drop=True)

# Display the finished dataframe (like the authors)
all_vars_vol.head()

Building DKT summed & normalized GrayVol features for all participants...


IntProgress(value=0, max=1601)

Feature columns: 33 summed, 33 normalized; + total_dkt_grayvol


,participant_id,p_factor,total_dkt_grayvol,sum_grayvol_caudalanteriorcingulate,sum_grayvol_caudalmiddlefrontal,sum_grayvol_cuneus,sum_grayvol_entorhinal,sum_grayvol_fusiform,sum_grayvol_inferiorparietal,sum_grayvol_inferiortemporal,...,norm_grayvol_rostralmiddlefrontal,norm_grayvol_superiorfrontal,norm_grayvol_superiorparietal,norm_grayvol_superiortemporal,norm_grayvol_supramarginal,norm_grayvol_transversetemporal,sum_grayvol_frontalpole,norm_grayvol_frontalpole,sum_grayvol_temporalpole,norm_grayvol_temporalpole
0,1000393599,0.589907,541205.0,5585.0,13717.0,11162.0,5090.0,17342.0,32167.0,25353.0,...,0.046450,0.113667,0.042239,0.074451,0.040988,0.004608,NaN,NaN,NaN,NaN
1,1001970838,-0.659061,489732.0,4389.0,12242.0,8171.0,2826.0,15497.0,27638.0,22427.0,...,0.048692,0.116123,0.042107,0.074002,0.047534,0.004153,NaN,NaN,NaN,NaN
2,1007995238,-1.608375,526299.0,5477.0,14809.0,8150.0,3677.0,21217.0,36134.0,24934.0,...,0.048746,0.102833,0.044271,0.066384,0.045541,0.003755,NaN,NaN,NaN,NaN
3,1011497669,-1.233807,535375.0,5906.0,15392.0,7839.0,3272.0,17358.0,35416.0,25016.0,...,0.054864,0.100963,0.044948,0.063684,0.046203,0.003396,NaN,NaN,NaN,NaN
4,1017092387,-0.923100,484183.0,5890.0,14017.0,8161.0,3420.0,13860.0,23049.0,23166.0,...,0.048556,0.117342,0.049242,0.062346,0.042796,0.004302,NaN,NaN,NaN,NaN


In [42]:
# Cell 6_AI: Build AI features for all participants (like Cell 6 but using AI)

print("Building DKT AI GrayVol features for all participants...")

from ipywidgets import IntProgress
from IPython.display import display

rows_ai = []
prog = IntProgress(min=0, max=len(all_data))
display(prog)

for _, row in all_data.iterrows():
    pid = str(row['participant_id'])
    pval = row.get('p_factor', np.nan)
    try:
        s = load_dkt_ai_grayvol(pid)
    except FileNotFoundError:
        s = pd.Series(dtype=float)

    rec = {'participant_id': pid, 'p_factor': pval}
    rec.update(s.to_dict())
    rows_ai.append(rec)
    prog.value += 1

all_vars_ai = pd.DataFrame(rows_ai)

# Split like authors
train_vars_ai = all_vars_ai[~np.isnan(all_vars_ai['p_factor'])].reset_index(drop=True)
test_vars_ai  = all_vars_ai[ np.isnan(all_vars_ai['p_factor'])].reset_index(drop=True)

print("AI feature columns:", sum(c.startswith('ai_grayvol_') for c in all_vars_ai.columns))
all_vars_ai.head()


Building DKT AI GrayVol features for all participants...


IntProgress(value=0, max=1601)

AI feature columns: 33


,participant_id,p_factor,ai_grayvol_caudalanteriorcingulate,ai_grayvol_caudalmiddlefrontal,ai_grayvol_cuneus,ai_grayvol_entorhinal,ai_grayvol_fusiform,ai_grayvol_inferiorparietal,ai_grayvol_inferiortemporal,ai_grayvol_insula,...,ai_grayvol_precuneus,ai_grayvol_rostralanteriorcingulate,ai_grayvol_rostralmiddlefrontal,ai_grayvol_superiorfrontal,ai_grayvol_superiorparietal,ai_grayvol_superiortemporal,ai_grayvol_supramarginal,ai_grayvol_transversetemporal,ai_grayvol_frontalpole,ai_grayvol_temporalpole
0,1000393599,0.589907,-0.250850,-0.025005,-0.030819,-0.066405,0.056626,0.159076,0.063622,0.003288,...,0.029334,-0.279949,0.007836,0.045760,0.051531,-0.035316,0.015282,-0.140337,NaN,NaN
1,1001970838,-0.659061,-0.137845,0.072047,-0.040020,-0.074310,-0.027037,0.099356,0.012886,0.011191,...,0.057381,-0.015019,-0.091839,0.062266,0.059599,-0.063326,-0.028137,-0.186824,NaN,NaN
2,1007995238,-1.608375,-0.226219,-0.018840,-0.040245,-0.132989,0.006457,0.083080,0.012593,0.039840,...,0.056477,-0.116896,0.039641,0.036234,0.023863,-0.042017,0.010514,-0.072874,NaN,NaN
3,1011497669,-1.233807,-0.082289,-0.116424,-0.107794,0.000000,-0.002189,0.119438,-0.002478,0.040342,...,-0.006547,-0.086276,-0.029381,0.050672,0.024850,-0.040563,-0.041397,-0.129813,NaN,NaN
4,1017092387,-0.923100,-0.068930,-0.011486,0.044480,-0.028655,-0.026407,0.078832,0.008547,-0.009399,...,-0.003602,-0.162029,-0.026967,0.030150,-0.023740,-0.035148,-0.048984,-0.059049,NaN,NaN


In [12]:
# Cell 7 — Load QC (robust), build drop lists, and filter train/test tables
# - Drops 9 known-bad participants from BOTH train & test
# - Drops QC == 'Fail' participants from TRAIN ONLY

from rbclib import RBCPath
from pathlib import Path
import pandas as pd
import numpy as np

def load_pnc_t1_qc_robust(local_cache_dir=(Path.home() / 'cache')):
    """
    Load the PNC structural T1 QC table, trying multiple RBC paths and a GitHub-raw fallback.
    Returns a dataframe with columns including:
      - participant_id  (string without 'sub-')
      - qc_determination (manual decision text)
      - qc_failed (boolean we set here: True if determination indicates fail/exclude)
    """
    if local_cache_dir is not None:
        Path(local_cache_dir).mkdir(exist_ok=True, parents=True)

    # Candidates (some environments only mirror subtrees)
    rbc_candidates = [
        ('rbc://PNC_FreeSurfer',            'study-PNC_desc-T1_qc.tsv'),
        ('rbc://PNC_FreeSurfer/freesurfer', 'study-PNC_desc-T1_qc.tsv'),
        ('rbc://PNC_BIDS/phenotype',        'study-PNC_desc_T1_qc.tsv'),  # alt naming in some docs
    ]

    qc = None
    last_err = None
    for root, name in rbc_candidates:
        try:
            tsv_path = RBCPath(root, local_cache_dir=local_cache_dir) / name
            with tsv_path.open('r') as f:
                qc = pd.read_csv(f, sep='\t')
            break
        except Exception as e:
            last_err = e
            qc = None

    if qc is None:
        # Fallback to GitHub raw
        gh_raw = "https://raw.githubusercontent.com/ReproBrainChart/PNC_FreeSurfer/main/study-PNC_desc-T1_qc.tsv"
        qc = pd.read_csv(gh_raw, sep='\t')

    # Harmonize IDs
    if 'participant_id' not in qc.columns:
        if 'subject_id' in qc.columns:
            qc['participant_id'] = qc['subject_id'].astype(str).str.replace('^sub-', '', regex=True).str.strip()
        else:
            # try to find any column containing 'sub-'
            sub_cols = [c for c in qc.columns if qc[c].astype(str).str.contains('^sub-').any()]
            if sub_cols:
                qc['participant_id'] = qc[sub_cols[0]].astype(str).str.replace('^sub-', '', regex=True).str.strip()
            else:
                raise ValueError("QC table lacks recognizable subject/participant ID columns.")

    # Determine failures from the manual decision field
    # Prefer 'qc_determination' if present; otherwise create a neutral column.
    if 'qc_determination' in qc.columns:
        decisions = qc['qc_determination'].astype(str).str.strip().str.lower()
        qc['qc_failed'] = decisions.isin({'fail', 'failed', 'exclude', 'excluded'})
    else:
        qc['qc_failed'] = False  # if column missing, default to no failures

    return qc

# --- Load QC and build the failure set for TRAIN ---
qc = load_pnc_t1_qc_robust()

qc_fail_participants = set(
    qc.loc[qc['qc_failed'] == True, 'participant_id'].astype(str).unique().tolist()
)

print(f"QC table loaded. Total rows: {len(qc)}")
print(f"QC fails detected: {len(qc_fail_participants)}")

# --- Build global & training-only drop sets ---
# From Cell 2: participants_to_drop is your fixed list of 9 IDs to remove everywhere
drop_ids_global   = set(str(x) for x in participants_to_drop)         # drop from BOTH train & test
drop_ids_training = drop_ids_global.union(qc_fail_participants)       # extra QC fails ONLY for train

# --- Apply drops to the feature tables produced in Cell 6 ---
def _drop_ids(df, ids):
    if df is None or len(df) == 0:
        return df
    out = df.copy()
    out['participant_id'] = out['participant_id'].astype(str)
    return out[~out['participant_id'].isin(ids)].reset_index(drop=True)

# all_vars_vol: drop only your 9 global IDs (keep QC fails for reference)
all_vars_vol_f   = _drop_ids(all_vars_vol, drop_ids_global)
train_vars_vol_f = _drop_ids(train_vars_vol, drop_ids_training)
test_vars_vol_f  = _drop_ids(test_vars_vol,  drop_ids_global)

# --- Quick report ---
def _nunique(df): return int(df['participant_id'].nunique()) if df is not None and 'participant_id' in df.columns else 0

print("\nCounts after filtering:")
print("  All (post global drops):", _nunique(all_vars_vol_f))
print("  Train (post global + QC):", _nunique(train_vars_vol_f))
print("  Test  (post global only):", _nunique(test_vars_vol_f))

# Keep handy lists of feature columns for later modeling
sum_cols_f  = [c for c in all_vars_vol_f.columns  if c.startswith('sum_grayvol_')]
norm_cols_f = [c for c in all_vars_vol_f.columns  if c.startswith('norm_grayvol_')]
all_feat_cols = sum_cols_f + norm_cols_f + (['total_dkt_grayvol'] if 'total_dkt_grayvol' in all_vars_vol_f.columns else [])

print("\nFeature columns prepared:")
print(f"  Summed: {len(sum_cols_f)} | Normalized: {len(norm_cols_f)} | Has total_dkt_grayvol: {'total_dkt_grayvol' in all_vars_vol_f.columns}")

# These filtered tables will be the inputs for the next steps (demographics merge + RF CV):
# - all_vars_vol_f   (combined participants; global drops applied)
# - train_vars_vol_f (training only; global + QC drops)
# - test_vars_vol_f  (test only; global drops applied)


QC table loaded. Total rows: 1592
QC fails detected: 8

Counts after filtering:
  All (post global drops): 1592
  Train (post global + QC): 1054
  Test  (post global only): 532

Feature columns prepared:
  Summed: 33 | Normalized: 33 | Has total_dkt_grayvol: True


In [43]:
# Cell 7_AI: Filter AI tables using your existing drop sets

def _ensure_set(x):
    return set(str(v) for v in x) if x is not None else set()

drop_ids_global   = _ensure_set(participants_to_drop)                   # from Cell 2
drop_ids_training = drop_ids_global.union(_ensure_set(globals().get('qc_fail_participants', set())))

def _drop_ids(df, ids):
    out = df.copy()
    out['participant_id'] = out['participant_id'].astype(str)
    return out[~out['participant_id'].isin(ids)].reset_index(drop=True)

all_vars_ai_f   = _drop_ids(all_vars_ai, drop_ids_global)
train_vars_ai_f = _drop_ids(train_vars_ai, drop_ids_training)
test_vars_ai_f  = _drop_ids(test_vars_ai,  drop_ids_global)

def _nunique(df): return int(df['participant_id'].nunique()) if 'participant_id' in df.columns else 0
print("Counts after AI filtering:")
print("  All:",   _nunique(all_vars_ai_f))
print("  Train:", _nunique(train_vars_ai_f))
print("  Test:",  _nunique(test_vars_ai_f))


Counts after AI filtering:
  All: 1592
  Train: 1054
  Test: 532


In [17]:
# Cell 8 — Demographics update:
#   • single_parent → categorical ('Yes'/'No')
#   • remove 'wave' from features
#   • keep prior logic (drop BMI, fill parent_max_education to min level)
#   • auto-drop low-coverage brain features (e.g., frontalpole/temporalpole)

import pandas as pd
import numpy as np

# ---- Parent education categories and ordinal mapping ----
EDU_ORDER = [
    "No/incomplete primary",
    "Complete primary",
    "Complete secondary",
    "Complete tertiary",
]
EDU_SCORE = {label: i for i, label in enumerate(EDU_ORDER)}  # 0..3

def _norm_parent_label(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    return s if s in EDU_SCORE else np.nan

def _edu_score(x):
    return EDU_SCORE.get(x, np.nan)

# ---- Build demographics with derived variables ----
# NOTE: 'bmi' intentionally omitted; 'wave' removed per request
demo_source_cols = [
    'age','sex','race','ethnicity','handedness',
    'participant_education','parent_1_education','parent_2_education',
    'study_site','study'
]
demo = all_data[['participant_id'] + [c for c in demo_source_cols if c in all_data.columns]].copy()
demo['participant_id'] = demo['participant_id'].astype(str)

# Parent labels normalized to the four allowed values
p1 = demo['parent_1_education'].apply(_norm_parent_label) if 'parent_1_education' in demo.columns else pd.Series(np.nan, index=demo.index)
p2 = demo['parent_2_education'].apply(_norm_parent_label) if 'parent_2_education' in demo.columns else pd.Series(np.nan, index=demo.index)

# Max parent education (by ordinal score)
p1_score = p1.apply(_edu_score)
p2_score = p2.apply(_edu_score)
use_p1 = p1_score.fillna(-np.inf) >= p2_score.fillna(-np.inf)
demo['parent_max_education'] = np.where(use_p1, p1, p2)

# single_parent: 1 if either OR both parent entries are missing; else 0 → then cast to categorical
single_parent_num = np.where(p1.isna() | p2.isna(), 1.0, 0.0)
demo['single_parent'] = pd.Series(single_parent_num, index=demo.index).map({1.0: 'Yes', 0.0: 'No'}).astype('category')

# Fill missing parent_max_education with MIN level
demo['parent_max_education'] = demo['parent_max_education'].fillna(EDU_ORDER[0]).astype('category')

# Keep only intended demographics (no BMI, no wave)
demo_cols = [
    'age','sex','race','ethnicity','handedness',
    'participant_education','parent_max_education','single_parent',
    'study_site','study'
]
demo = demo[['participant_id'] + [c for c in demo_cols if c in demo.columns]]

# ---- Merge demographics into the filtered feature tables from Cell 7 ----
def _merge_demo(feat_df):
    out = feat_df.copy()
    out['participant_id'] = out['participant_id'].astype(str)
    return out.merge(demo, on='participant_id', how='left')

train_model_df = _merge_demo(train_vars_vol_f)
test_model_df  = _merge_demo(test_vars_vol_f)

# ---- Auto-drop low-coverage brain features (on TRAIN only) ----
norm_cols_all = [c for c in train_model_df.columns if c.startswith('norm_grayvol_')]
coverage = 1.0 - train_model_df[norm_cols_all].isna().mean()
keep_norm = coverage[coverage >= 0.95].index.tolist()
drop_norm = sorted(set(norm_cols_all) - set(keep_norm))

feature_cols_brain = keep_norm + (['total_dkt_grayvol'] if 'total_dkt_grayvol' in train_model_df.columns else [])

print(f"Train rows: {len(train_model_df)} | Test rows: {len(test_model_df)}")
print(f"Brain features kept: {len(feature_cols_brain)}")
if drop_norm:
    print("Dropped low-coverage features:", [c.replace('norm_grayvol_', '') for c in drop_norm][:20])

# Final feature list to feed the model (used by Cell 9)
keep_cols = feature_cols_brain + [c for c in demo_cols if c in train_model_df.columns]
print("Example kept brain features:", [c for c in feature_cols_brain if c.startswith('norm_grayvol_')][:5])

# ---- Missingness BEFORE any imputation (among KEPT columns) ----
miss_train = train_model_df[keep_cols + ['p_factor']].isna().sum().sort_values(ascending=False)
miss_test  = test_model_df[keep_cols].isna().sum().sort_values(ascending=False)

print("\nMissing values in TRAIN (per kept column):")
print(miss_train[miss_train > 0].head(40))
print("\nMissing values in TEST (per kept column):")
print(miss_test[miss_test > 0].head(40))

rows_with_any_missing_train = train_model_df[keep_cols + ['p_factor']].isna().any(axis=1).sum()
rows_with_any_missing_test  = test_model_df[keep_cols].isna().any(axis=1).sum()
print(f"\nRows with ANY missing among KEPT columns (TRAIN): {rows_with_any_missing_train} / {len(train_model_df)}")
print(f"Rows with ANY missing among KEPT columns (TEST):  {rows_with_any_missing_test} / {len(test_model_df)}")


Train rows: 1054 | Test rows: 532
Brain features kept: 32
Dropped low-coverage features: ['frontalpole', 'temporalpole']
Example kept brain features: ['norm_grayvol_caudalanteriorcingulate', 'norm_grayvol_caudalmiddlefrontal', 'norm_grayvol_cuneus', 'norm_grayvol_entorhinal', 'norm_grayvol_fusiform']

Missing values in TRAIN (per kept column):
Series([], dtype: int64)

Missing values in TEST (per kept column):
Series([], dtype: int64)

Rows with ANY missing among KEPT columns (TRAIN): 0 / 1054
Rows with ANY missing among KEPT columns (TEST):  0 / 532


In [45]:
# Cell 8_AI — Demographics + AI features (fixed parent_max_education construction)
# - Excludes BMI and wave
# - single_parent as categorical ('Yes'/'No')
# - parent_max_education = max(parent_1/2 by ordinal; fill to min level if missing)
# - Auto-drops low-coverage AI features (>=95% coverage kept)
# - Produces: train_model_df_ai, test_model_df_ai, feature_cols_brain_ai, keep_cols_ai

import pandas as pd
import numpy as np

# --- Parent education mapping (exact labels you provided) ---
EDU_ORDER = [
    "No/incomplete primary",
    "Complete primary",
    "Complete secondary",
    "Complete tertiary",
]
EDU_SCORE = {label: i for i, label in enumerate(EDU_ORDER)}  # 0..3

def _norm_parent_label(x):
    """Return the label only if it is one of the four known values; else NaN."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    return s if s in EDU_SCORE else np.nan

def _edu_score(x):
    """Map label -> ordinal score (0..3); NaN if unknown/missing."""
    return EDU_SCORE.get(x, np.nan)

# --- Build demographics (no BMI, no wave) ---
demo_source_cols = [
    'age','sex','race','ethnicity','handedness',
    'participant_education','parent_1_education','parent_2_education',
    'study_site','study'
]
demo = all_data[['participant_id'] + [c for c in demo_source_cols if c in all_data.columns]].copy()
demo['participant_id'] = demo['participant_id'].astype(str)

# Normalize parent labels to the four allowed values
p1 = demo['parent_1_education'].apply(_norm_parent_label) if 'parent_1_education' in demo.columns else pd.Series(np.nan, index=demo.index)
p2 = demo['parent_2_education'].apply(_norm_parent_label) if 'parent_2_education' in demo.columns else pd.Series(np.nan, index=demo.index)

# Choose the higher-educated parent by ordinal score (ties -> parent_1; one missing -> the other; both missing -> NaN)
p1_score = p1.apply(_edu_score)
p2_score = p2.apply(_edu_score)
use_p1 = p1_score.fillna(-np.inf) >= p2_score.fillna(-np.inf)

# FIX: wrap np.where(...) in a Series before fillna/astype
parent_choice = pd.Series(np.where(use_p1, p1, p2), index=demo.index, name='parent_max_education')
demo['parent_max_education'] = parent_choice.fillna(EDU_ORDER[0]).astype('category')

# single_parent: 'Yes' if either/both parent entries are missing; else 'No' (as categorical)
single_parent_num = np.where(p1.isna() | p2.isna(), 1.0, 0.0)
demo['single_parent'] = pd.Series(single_parent_num, index=demo.index).map({1.0: 'Yes', 0.0: 'No'}).astype('category')

# Keep only intended demographics
demo_cols = [
    'age','sex','race','ethnicity','handedness',
    'participant_education','parent_max_education','single_parent',
    'study_site','study'
]
demo = demo[['participant_id'] + [c for c in demo_cols if c in demo.columns]]

# --- Merge demographics into the AI feature tables from Cell 7_AI ---
def _merge_demo(feat_df):
    out = feat_df.copy()
    out['participant_id'] = out['participant_id'].astype(str)
    return out.merge(demo, on='participant_id', how='left')

train_model_df_ai = _merge_demo(train_vars_ai_f)
test_model_df_ai  = _merge_demo(test_vars_ai_f)

# --- Auto-drop low-coverage AI features (computed on TRAIN only) ---
ai_cols_all = [c for c in train_model_df_ai.columns if c.startswith('ai_grayvol_')]
coverage = 1.0 - train_model_df_ai[ai_cols_all].isna().mean()
keep_ai = coverage[coverage >= 0.95].index.tolist()
drop_ai = sorted(set(ai_cols_all) - set(keep_ai))
if drop_ai:
    print("Dropped low-coverage AI features:", [c.replace('ai_grayvol_','') for c in drop_ai][:20])

# Final brain feature list for the AI path
feature_cols_brain_ai = keep_ai

# Columns to feed the model (AI features + demographics)
keep_cols_ai = feature_cols_brain_ai + [c for c in demo_cols if c in train_model_df_ai.columns]

print(f"AI Train rows: {len(train_model_df_ai)} | AI Test rows: {len(test_model_df_ai)}")
print(f"AI features kept: {len(feature_cols_brain_ai)}")
print("Example kept AI features:", feature_cols_brain_ai[:5])

# --- Missingness BEFORE any imputation (among KEPT columns) ---
miss_train_ai = train_model_df_ai[keep_cols_ai + ['p_factor']].isna().sum().sort_values(ascending=False)
miss_test_ai  = test_model_df_ai[keep_cols_ai].isna().sum().sort_values(ascending=False)

print("\nMissing in TRAIN (AI kept cols):")
print(miss_train_ai[miss_train_ai > 0].head(20))
print("\nMissing in TEST (AI kept cols):")
print(miss_test_ai[miss_test_ai > 0].head(20))

rows_with_any_missing_train = train_model_df_ai[keep_cols_ai + ['p_factor']].isna().any(axis=1).sum()
rows_with_any_missing_test  = test_model_df_ai[keep_cols_ai].isna().any(axis=1).sum()
print(f"\nRows with ANY missing among KEPT (TRAIN): {rows_with_any_missing_train} / {len(train_model_df_ai)}")
print(f"Rows with ANY missing among KEPT (TEST):  {rows_with_any_missing_test} / {len(test_model_df_ai)}")


Dropped low-coverage AI features: ['frontalpole', 'temporalpole']
AI Train rows: 1054 | AI Test rows: 532
AI features kept: 31
Example kept AI features: ['ai_grayvol_caudalanteriorcingulate', 'ai_grayvol_caudalmiddlefrontal', 'ai_grayvol_cuneus', 'ai_grayvol_entorhinal', 'ai_grayvol_fusiform']

Missing in TRAIN (AI kept cols):
Series([], dtype: int64)

Missing in TEST (AI kept cols):
Series([], dtype: int64)

Rows with ANY missing among KEPT (TRAIN): 0 / 1054
Rows with ANY missing among KEPT (TEST):  0 / 532


In [53]:
# Cell 8.5 — Unified feature switcher (AI ↔︎ Volume)
# Sets the canonical: train_model_df, test_model_df, keep_cols

import pandas as pd
import numpy as np

# Choose which feature set to use: 'ai' or 'vol'
feature_mode = 'vol'   # <-- set to 'ai' or 'vol'

# Require demographics from Cell 8/8_AI
assert 'demo' in globals() and 'demo_cols' in globals(), \
    "Missing demographics (demo/demo_cols). Run Cell 8 (or 8_AI) first."

def _merge_demo(feat_df):
    out = feat_df.copy()
    out['participant_id'] = out['participant_id'].astype(str)
    return out.merge(demo, on='participant_id', how='left')

if feature_mode == 'vol':
    # Need volume feature tables from Cells 6 & 7
    assert 'train_vars_vol_f' in globals() and 'test_vars_vol_f' in globals(), \
        "Missing volume feature tables. Run Cells 6 & 7."

    train_model_df_vol = _merge_demo(train_vars_vol_f)
    test_model_df_vol  = _merge_demo(test_vars_vol_f)

    # Auto-drop low-coverage normalized VOLUME features (train only)
    norm_cols_all = [c for c in train_model_df_vol.columns if c.startswith('norm_grayvol_')]
    coverage = 1.0 - train_model_df_vol[norm_cols_all].isna().mean()
    keep_norm = coverage[coverage >= 0.95].index.tolist()
    drop_norm = sorted(set(norm_cols_all) - set(keep_norm))

    feature_cols_brain_vol = keep_norm + (['total_dkt_grayvol'] if 'total_dkt_grayvol' in train_model_df_vol.columns else [])
    keep_cols_vol = feature_cols_brain_vol + [c for c in demo_cols if c in train_model_df_vol.columns]

    # Set canonical
    train_model_df = train_model_df_vol
    test_model_df  = test_model_df_vol
    keep_cols      = keep_cols_vol

    print("Switched to VOLUME features.")
    print(f"Train rows: {len(train_model_df)} | Test rows: {len(test_model_df)}")
    print(f"Brain features kept (vol): {len(feature_cols_brain_vol)}")
    if drop_norm:
        print("Dropped low-coverage (vol):", [c.replace('norm_grayvol_', '') for c in drop_norm][:20])

elif feature_mode == 'ai':
    # Need AI feature tables from Cells 6_AI & 7_AI
    assert 'train_vars_ai_f' in globals() and 'test_vars_ai_f' in globals(), \
        "Missing AI feature tables. Run Cells 6_AI & 7_AI."

    train_model_df_ai = _merge_demo(train_vars_ai_f)
    test_model_df_ai  = _merge_demo(test_vars_ai_f)

    # Auto-drop low-coverage AI features (train only)
    ai_cols_all = [c for c in train_model_df_ai.columns if c.startswith('ai_grayvol_')]
    coverage = 1.0 - train_model_df_ai[ai_cols_all].isna().mean()
    keep_ai = coverage[coverage >= 0.95].index.tolist()
    drop_ai = sorted(set(ai_cols_all) - set(keep_ai))

    feature_cols_brain_ai = keep_ai
    keep_cols_ai = feature_cols_brain_ai + [c for c in demo_cols if c in train_model_df_ai.columns]

    # Set canonical
    train_model_df = train_model_df_ai
    test_model_df  = test_model_df_ai
    keep_cols      = keep_cols_ai

    print("Switched to AI features.")
    print(f"Train rows: {len(train_model_df)} | Test rows: {len(test_model_df)}")
    print(f"Brain features kept (ai): {len(feature_cols_brain_ai)}")
    if drop_ai:
        print("Dropped low-coverage (ai):", [c.replace('ai_grayvol_', '') for c in drop_ai][:20])

else:
    raise ValueError("feature_mode must be 'vol' or 'ai'")

# Sanity: show a few feature names and confirm mode
print("First 5 feature columns:", keep_cols[:5])
print("Any AI features in keep_cols? ", any(c.startswith('ai_grayvol_') for c in keep_cols))
print("Any VOLUME features in keep_cols? ", any(c.startswith('norm_grayvol_') for c in keep_cols))


Switched to VOLUME features.
Train rows: 1054 | Test rows: 532
Brain features kept (vol): 32
Dropped low-coverage (vol): ['frontalpole', 'temporalpole']
First 5 feature columns: ['norm_grayvol_caudalanteriorcingulate', 'norm_grayvol_caudalmiddlefrontal', 'norm_grayvol_cuneus', 'norm_grayvol_entorhinal', 'norm_grayvol_fusiform']
Any AI features in keep_cols?  False
Any VOLUME features in keep_cols?  True


In [54]:
# Inspector — which columns are categorical vs numerical?

import pandas as pd
import numpy as np

X_train = train_model_df[keep_cols].copy()

# Infer types the way scikit-learn will use them
cat_cols = sorted(X_train.select_dtypes(include=['object', 'category']).columns.tolist())
num_cols = sorted(X_train.select_dtypes(include=['number', 'bool']).columns.tolist())

print(f"# categorical: {len(cat_cols)}")
print(cat_cols)
print("\n# numerical:   {0}".format(len(num_cols)))
print(num_cols)

# Quick dtype summary
print("\nDtype counts in X_train:")
print(X_train.dtypes.value_counts())

# Peek at levels of each categorical (first 8 uniques)
print("\nCategorical columns (unique values up to 8 each):")
for c in cat_cols:
    vals = X_train[c].dropna().unique()
    print(f"  {c}: {len(vals)} levels -> {vals[:8]}")

# categorical: 9
['ethnicity', 'handedness', 'parent_max_education', 'participant_education', 'race', 'sex', 'single_parent', 'study', 'study_site']

# numerical:   33
['age', 'norm_grayvol_caudalanteriorcingulate', 'norm_grayvol_caudalmiddlefrontal', 'norm_grayvol_cuneus', 'norm_grayvol_entorhinal', 'norm_grayvol_fusiform', 'norm_grayvol_inferiorparietal', 'norm_grayvol_inferiortemporal', 'norm_grayvol_insula', 'norm_grayvol_isthmuscingulate', 'norm_grayvol_lateraloccipital', 'norm_grayvol_lateralorbitofrontal', 'norm_grayvol_lingual', 'norm_grayvol_medialorbitofrontal', 'norm_grayvol_middletemporal', 'norm_grayvol_paracentral', 'norm_grayvol_parahippocampal', 'norm_grayvol_parsopercularis', 'norm_grayvol_parsorbitalis', 'norm_grayvol_parstriangularis', 'norm_grayvol_pericalcarine', 'norm_grayvol_postcentral', 'norm_grayvol_posteriorcingulate', 'norm_grayvol_precentral', 'norm_grayvol_precuneus', 'norm_grayvol_rostralanteriorcingulate', 'norm_grayvol_rostralmiddlefrontal', 'norm_grayv

In [56]:
# Cell 9 — Random Forest with imputers (leakage-safe),
# (Fixed OneHotEncoder arg for scikit-learn >=1.2 via a compatibility shim.)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
import numpy as np
import pandas as pd

# Build X/y from Cell 8 outputs
y_train = pd.to_numeric(train_model_df['p_factor'], errors='coerce')
X_train = train_model_df[keep_cols].copy()

# Identify column types (as sklearn will)
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X_train.select_dtypes(include=['number', 'bool']).columns.tolist()

# --- OneHotEncoder compatibility shim ---
try:
    # scikit-learn >= 1.2
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    # scikit-learn <= 1.1
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Preprocess INSIDE the pipeline to avoid leakage:
#  - Categorical: impute most_frequent, then one-hot (ignore unseen categories)
#  - Numeric:     impute median (covers any missing that may appear in test)
pre = ColumnTransformer([
    ('cat', Pipeline(steps=[
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('onehot', ohe)
    ]), cat_cols),
    ('num', SimpleImputer(strategy='median'), num_cols),
])

# Random Forest (constrained to reduce overfitting)
rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=4,
    min_samples_leaf=3,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=42,
)

pipe = Pipeline([('prep', pre), ('rf', rf)])

# 5-fold CV on TRAIN ONLY (honest generalization estimate)
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_r2  = cross_val_score(pipe, X_train, y_train, cv=cv, scoring='r2')
cv_mae = -cross_val_score(pipe, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error')

print("Random Forest (normalized DKT volumes + total + demographics)")
print(f"CV R^2 (mean ± sd):  {cv_r2.mean():.3f} ± {cv_r2.std():.3f}")
print(f"CV MAE (mean ± sd): {cv_mae.mean():.3f} ± {cv_mae.std():.3f}")

# Fit once on full training set for context (Training R^2 and OOB R^2)
pipe.fit(X_train, y_train)
print(f"Training R^2: {pipe.score(X_train, y_train):.3f}")
print(f"OOB R^2:      {pipe.named_steps['rf'].oob_score_:.3f}")

# Keep 'pipe' fitted for the next cell (test predictions)


Random Forest (normalized DKT volumes + total + demographics)
CV R^2 (mean ± sd):  0.065 ± 0.032
CV MAE (mean ± sd): 0.751 ± 0.031
Training R^2: 0.263
OOB R^2:      0.064


In [61]:
# Cell 9b — Tiny sweep with progress bar, auto-detect dataset changes, lock best, fit global `pipe`

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from ipywidgets import IntProgress
from IPython.display import display

# === Configure your grid here ===
depths = [3, 4, 5]
leaves = [2, 3, 4]
force_resweep = True   # set True to force a new sweep

# Build train matrices (from previous cells)
y_train = pd.to_numeric(train_model_df['p_factor'], errors='coerce')
X_train = train_model_df[keep_cols].copy()

# Column typing
cat_cols = X_train.select_dtypes(include=['object','category']).columns.tolist()
num_cols = X_train.select_dtypes(include=['number','bool']).columns.tolist()

# OneHotEncoder compatibility shim
try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

def make_preprocessor():
    return ColumnTransformer([
        ('cat', Pipeline([
            ('impute', SimpleImputer(strategy='most_frequent')),
            ('onehot', ohe)
        ]), cat_cols),
        ('num', SimpleImputer(strategy='median'), num_cols),
    ])

# --- Decide whether to sweep (detect grid OR dataset changes) ---
grid_stamp = (tuple(depths), tuple(leaves))
# dataset stamp: feature columns (order matters) + #rows + quick flag of feature type
is_ai = any(c.startswith('ai_grayvol_') for c in keep_cols)
dataset_stamp = (tuple(keep_cols), X_train.shape[0], is_ai)

if '_rf_sweep_cache' not in globals():
    _rf_sweep_cache = {}

need_sweep = (
    force_resweep
    or ('res_df' not in _rf_sweep_cache)
    or (_rf_sweep_cache.get('grid_stamp') != grid_stamp)
    or (_rf_sweep_cache.get('dataset_stamp') != dataset_stamp)
)

if need_sweep:
    total = len(depths) * len(leaves)
    prog = IntProgress(min=0, max=total, description="Tuning RF…")
    display(prog)

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    results = []
    for depth in depths:
        for leaf in leaves:
            rf = RandomForestRegressor(
                n_estimators=1000,
                max_depth=depth,
                min_samples_leaf=leaf,
                bootstrap=True,
                oob_score=True,
                n_jobs=-1,
                random_state=42,
            )
            pipe_tmp = Pipeline([('prep', make_preprocessor()), ('rf', rf)])

            # CV R^2
            cv_r2 = cross_val_score(pipe_tmp, X_train, y_train, cv=cv, scoring='r2')

            # Fit once for OOB
            pipe_tmp.fit(X_train, y_train)

            results.append({
                'max_depth': depth,
                'min_samples_leaf': leaf,
                'cv_r2_mean': float(cv_r2.mean()),
                'cv_r2_sd':   float(cv_r2.std()),
                'oob_r2':     float(pipe_tmp.named_steps['rf'].oob_score_),
            })
            prog.value += 1

    prog.bar_style = 'success'
    res_df = pd.DataFrame(results).sort_values('cv_r2_mean', ascending=False).reset_index(drop=True)
    _rf_sweep_cache['res_df'] = res_df
    _rf_sweep_cache['grid_stamp'] = grid_stamp
    _rf_sweep_cache['dataset_stamp'] = dataset_stamp

    print("Tiny sweep results (sorted by CV R^2 mean):")
    print(res_df.to_string(index=False, float_format=lambda v: f"{v:.3f}"))
else:
    res_df = _rf_sweep_cache['res_df']
    print("Using cached sweep for grid & dataset.")

# --- Lock in best hyperparameters from res_df and fit global `pipe` ---
best = res_df.iloc[0]
best_depth = int(best['max_depth'])
best_leaf  = int(best['min_samples_leaf'])
print(f"\nRefitting with best config: depth={best_depth}, min_samples_leaf={best_leaf}")

pre = make_preprocessor()
rf = RandomForestRegressor(
    n_estimators=1000,
    max_depth=best_depth,
    min_samples_leaf=best_leaf,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=42,
)
pipe = Pipeline([('prep', pre), ('rf', rf)])
pipe.fit(X_train, y_train)
print(f"Training R^2: {pipe.score(X_train, y_train):.3f}")
print(f"OOB R^2:      {pipe.named_steps['rf'].oob_score_:.3f}")


IntProgress(value=0, description='Tuning RF…', max=9)

Tiny sweep results (sorted by CV R^2 mean):
 max_depth  min_samples_leaf  cv_r2_mean  cv_r2_sd  oob_r2
         4                 2       0.065     0.032   0.063
         4                 3       0.065     0.032   0.063
         3                 2       0.064     0.030   0.062
         5                 2       0.064     0.034   0.063
         3                 4       0.064     0.030   0.063
         4                 4       0.064     0.033   0.063
         3                 3       0.064     0.030   0.063
         5                 3       0.064     0.035   0.063
         5                 4       0.063     0.036   0.062

Refitting with best config: depth=4, min_samples_leaf=2
Training R^2: 0.261
OOB R^2:      0.063


In [62]:
# Cell 10 — Predict p_factor for TEST set and save to CSV

import pandas as pd
from pathlib import Path

# Use the fitted pipeline `pipe` from Cell 9 and the same feature list `keep_cols` from Cell 8
X_test = test_model_df[keep_cols].copy()

# Predict p_factor for the test participants
y_test_pred = pipe.predict(X_test)

# Build predictions dataframe (participant_id + predicted p_factor)
test_preds = pd.DataFrame({
    'participant_id': test_model_df['participant_id'].astype(str).values,
    'p_factor': y_test_pred
})

# Save to CSV (no overwrite of any provided files)
out_path = Path.cwd() / "test_p_factor_predictions.csv"
test_preds.to_csv(out_path, index=False)

print("Wrote predictions to:", out_path)
print("Shape:", test_preds.shape)
display(test_preds.head())

Wrote predictions to: /home/jovyan/chn-hackathon-2025/test_p_factor_predictions.csv
Shape: (532, 2)


,participant_id,p_factor
0,1000881804,-0.386976
1,100527940,-0.477904
2,1006151876,0.001385
3,1012530688,-0.636700
4,1030193285,-0.493121
